<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/Copatrade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ta
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import ta # Biblioteca de Análise Técnica

# --- Configuração dos Ativos ---
configuracao_ativos = {
    "Mini Índice":    {"ticker": "^BVSP", "max_contratos": 10},
    "Mini Dólar":     {"ticker": "BRL=X", "max_contratos": 5},
    "Bitcoin Futuro": {"ticker": "BTC-USD", "max_contratos": 20},
    "Ether Futuro":   {"ticker": "ETH-USD", "max_contratos": 10},
    "Solana Futuro":  {"ticker": "SOL-USD", "max_contratos": 10},
    "Ouro Futuro":    {"ticker": "GC=F", "max_contratos": 5},
    "Euro Futuro":    {"ticker": "EURBRL=X", "max_contratos": 5}
}

# --- Função de Análise Enriquecida ---
def analisar_ativo(ticker):
    """
    Busca os dados de um ativo, calcula múltiplos indicadores (MM, RSI, MACD, ATR)
    e retorna uma recomendação baseada na confluência de sinais.
    """
    try:
        # Adicionado auto_adjust=False para silenciar o aviso e garantir a obtenção das colunas High/Low
        dados = yf.download(ticker, period="90d", interval="1d", progress=False, auto_adjust=False)

        if dados.empty or len(dados) < 30:
            return {"Preço": "N/A", "RSI": "N/A", "ATR": "N/A", "Recomendação": "Dados insuficientes"}

        # --- CORREÇÃO: Força a conversão para o tipo 'float' para garantir compatibilidade ---
        close_prices = dados['Close'].squeeze().astype(float)
        high_prices = dados['High'].squeeze().astype(float)
        low_prices = dados['Low'].squeeze().astype(float)

        # --- Indicadores Base ---
        ultimo_preco = close_prices.iloc[-1]
        mm_curta = close_prices.ewm(span=9, adjust=False).mean().iloc[-1]
        mm_longa = close_prices.ewm(span=21, adjust=False).mean().iloc[-1]
        rsi = ta.momentum.RSIIndicator(close_prices, window=14).rsi().iloc[-1]

        # --- INDICADOR MACD ---
        macd_indicador = ta.trend.MACD(close_prices)
        macd_linha = macd_indicador.macd().iloc[-1]
        macd_sinal = macd_indicador.macd_signal().iloc[-1]

        # --- INDICADOR ATR (VOLATILIDADE) ---
        atr = ta.volatility.AverageTrueRange(
            high=high_prices,
            low=low_prices,
            close=close_prices,
            window=14
        ).average_true_range().iloc[-1]

        # --- LÓGICA DE RECOMENDAÇÃO (CONFLUÊNCIA) ---
        recomendacao = "NEUTRO / MONITORAR"

        if (mm_curta > mm_longa) and (macd_linha > macd_sinal) and (rsi > 50 and rsi < 70):
            recomendacao = "COMPRA FORTE (Tendência e Momentum de Alta)"
        elif (mm_curta < mm_longa) and (macd_linha < macd_sinal) and (rsi < 50 and rsi > 30):
            recomendacao = "VENDA FORTE (Tendência e Momentum de Baixa)"
        elif (rsi < 30) and (macd_linha > macd_sinal):
            recomendacao = "MONITORAR COMPRA (Possível Reversão de Baixa)"
        elif (rsi > 70) and (macd_linha < macd_sinal):
            recomendacao = "MONITORAR VENDA (Possível Reversão de Alta)"

        return {
            "Preço": f"{ultimo_preco:.2f}",
            "RSI": f"{rsi:.2f}",
            "ATR": f"{atr:.2f}",
            "Recomendação": recomendacao
        }
    except Exception as e:
        return {"Preço": "N/A", "RSI": "N/A", "ATR": "N/A", "Recomendação": f"ERRO: {str(e)}"}

# --- Roda a Análise e Exibe o Painel ---
if __name__ == "__main__":
    print("=========================================================")
    print("      ANALISTA DE ATIVOS - COPA TRADE BTG (V8 Robusto)")
    print(f"      Análise gerada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print("=========================================================")

    resultados = []
    for nome, config in configuracao_ativos.items():
        print(f"-> Analisando {nome} ({config['ticker']})...")
        analise = analisar_ativo(config['ticker'])
        resultados.append({
            "Ativo": nome,
            "Preço Atual": analise["Preço"],
            "Volatilidade (ATR)": analise["ATR"],
            "RSI(14)": analise["RSI"],
            "Recomendação de Ação": analise["Recomendação"]
        })

    painel = pd.DataFrame(resultados)

    print("\n--- PAINEL DE ANÁLISE TÉCNICA AVANÇADA ---")
    pd.set_option('display.max_colwidth', None)
    print(painel.to_string(index=False))
    print("\nLembre-se: Esta é uma ferramenta de apoio. Opere com consciência e gerencie seus riscos.")

      ANALISTA DE ATIVOS - COPA TRADE BTG (V8 Robusto)
      Análise gerada em: 06/10/2025 23:19:11
-> Analisando Mini Índice (^BVSP)...
-> Analisando Mini Dólar (BRL=X)...
-> Analisando Bitcoin Futuro (BTC-USD)...
-> Analisando Ether Futuro (ETH-USD)...
-> Analisando Solana Futuro (SOL-USD)...
-> Analisando Ouro Futuro (GC=F)...
-> Analisando Euro Futuro (EURBRL=X)...

--- PAINEL DE ANÁLISE TÉCNICA AVANÇADA ---
         Ativo Preço Atual Volatilidade (ATR) RSI(14)                        Recomendação de Ação
   Mini Índice   143608.08            1485.75   51.12                          NEUTRO / MONITORAR
    Mini Dólar        5.31               0.05   40.74                          NEUTRO / MONITORAR
Bitcoin Futuro   124898.38            2629.35   72.96                          NEUTRO / MONITORAR
  Ether Futuro     4694.39             168.46   63.46 COMPRA FORTE (Tendência e Momentum de Alta)
 Solana Futuro      233.49              10.90   57.73 COMPRA FORTE (Tendência e Momentum de Al